# Food Classification with CNN - Building a Restaurant Recommendation System

This assignment focuses on developing a deep learning-based food classification system using Convolutional Neural Networks (CNNs). You will build a model that can recognize different food categories and use it to return the food preferences of a user.

## Learning Objectives
- Implement CNNs for image classification
- Work with real-world food image datasets
- Build a preference-detector system

## Background: AI-Powered Food Preference Discovery

The system's core idea is simple:

1. Users upload 10 photos of dishes they enjoy
2. Your CNN classifies these images into the 91 categories
3. Based on these categories, the system returns the user's taste profile

Your task is to develop the core computer vision component that will power this detection engine.

You are given a training ("train" folder) and a test ("test" folder) dataset which have ~45k and ~22k samples respectively. For each one of the 91 classes there is a subdirectory containing the images of the respective class.

## Assignment Requirements

### Technical Requirements
- Implement your own pytorch CNN architecture for food image classification
- Use only the provided training dataset split for training
- Train the network from scratch ; No pretrained weights can be used
- Report test-accuracy after every epoch
- Report all hyperparameters of final model
- Use a fixed seed and do not use any CUDA-features that break reproducibility
- Use Pytorch 2.6

### Deliverables
1. Jupyter Notebook with CNN implementation, training code etc.
2. README file
3. Report (max 3 pages)

Submit your report, README and all code files as a single zip file named GROUP_[number]_NC2425_PA. The names and IDs of the group components must be mentioned in the README.
Do not include the dataset in your submission.

### Grading

1. Correct CNN implementation, training runs on the uni DSLab computers according to the README.MD instructions without ANY exceptions on the DSLab machines: 3pt
2. Perfect 1:1 reproducibility on DSLab machines: 1pt
3. Very clear github-repo-style README.MD with instructions for running the code: 1pt
4. Report: 1pt
5. Model test performance on test-set: interpolated from 30-80% test-accuracy: 0-3pt
6. Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt
7. Bonus point: use an LLM (API) to generate short description / profile of preferences of the simulated user

**If there is anything unclear about this assignment please post your question in the Brightspace discussions forum or send an email**


# Loading the datasets
The dataset is already split into a train and test set in the directories "train" and "test". 

In [11]:
### IMPORTS ###

import torch
import numpy as np
import torch.nn as nn
from time import time
import torch.nn.functional as F
from torchvision import datasets
from datetime import datetime as dt
from torchvision.transforms import v2
from torch.utils.data import DataLoader

In [12]:
### DEVICE ###
# Use the GPU to speed up the training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [13]:
### SEED FOR REPRODUCIBILITY ###
random_seed = 13
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

In [14]:
### HYPER-PARAMETERS ###

# Model training
workers = 8
batch_size = 16
num_epochs = 100 #50 I hope 100 is high enough
lr = 0.01
momentum = 0.75

In [15]:
### DATA LOADING ###
# This section loads the data from the train and test folders and applies the transformations
# All the images are resized to 256x256 pixels and normalized
# The training data is augmented with random transformations

# Define the transformations for the data
train_transform = v2.Compose([
    v2.Resize((256, 256)),
    v2.RandomApply([v2.TrivialAugmentWide()], p=0.3),
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = v2.Compose([
    v2.Resize((256, 256)),
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Collect the data and set the augmentation
train_dataset = datasets.ImageFolder(root='train', transform=train_transform)
test_dataset = datasets.ImageFolder(root='test', transform=test_transform) 

# Prepare the data loaders with the batches, shuffling, and workers
train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=workers)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=workers)


# CNN Implementation

In [16]:
### MODEL TOPOLOGY DEFINITION ###

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels, out_channels, 3, stride, 1),
                                    nn.BatchNorm2d(out_channels),
                                    nn.SiLU())
        
        self.layer2 = nn.Sequential(nn.Conv2d(out_channels, out_channels, 3, 1, 1),
                                    nn.BatchNorm2d(out_channels))
        
        self.proj = False
        if in_channels != out_channels or stride != 1:
            self.proj = True
            self.p_layer = nn.Conv2d(in_channels, out_channels, 1, stride)

    def forward(self, x):
        res = x
        out = self.layer1(x)
        out = self.layer2(out)
        if self.proj:
            res = self.p_layer(x)
        return nn.ReLU()(out+res)


class CNN(nn.Module):
    """
    Made to emulate the Residual Network.
    """
    def __init__(self, num_blocks=None):
        super().__init__()
        if num_blocks is None:
            num_blocks = [3, 4, 6, 3]
        self.layer1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(64),
                                    nn.MaxPool2d(3, 2, 1))          # 64

        self.layer2 = self._make_layer(64, 64, num_blocks[0], 1)
        self.layer3 = self._make_layer(64, 128, num_blocks[1])      # 32
        self.layer4 = self._make_layer(128, 256, num_blocks[2])     # 16
        self.layer5 = self._make_layer(256, 512, num_blocks[3])     # 8

        self.layer6 = nn.Sequential(nn.AvgPool2d(8),
                                    nn.Flatten(),
                                    nn.Dropout(0.25),
                                    nn.Linear(512, 91))
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        return F.softmax(x, dim=1)
    
    def _make_layer(self, in_channels, out_channels, blocks, stride=2):
        layers = [Block(in_channels, out_channels, stride)]
        for _ in range(1, blocks):
            layers.append(Block(out_channels, out_channels))
        return nn.Sequential(*layers)

# Training the model
Implement your training process below. Report the test-accuracy after every epoch for the training run of the final model.

Hint: before training your model make sure to reset the seed in the training cell, as otherwise the seed may have changed due to previous training runs in the notebook

Note: If you implement automatic hyperparameter tuning, split the train set into train and validation subsets for the objective function.

In [17]:
### TRAINING FUNCTIONS ###

def train(model, dataloader, optimizer, loss_fn):
    """
    One epoch of model training.
    """

    model.train()

    running_loss = 0.0

    for _, data in enumerate(dataloader):
        images, labels = data[0].to(device), data[1].to(device)

        # Forward propagate
        optimizer.zero_grad()
        outputs = model(images)


        # Back-propagate and optimize
        loss = loss_fn(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()


def accuracy(model, dataloader):
    """
    One test accuracy test.
    """
    model.eval()

    running_accuracy = 0.0
    for data in iter(dataloader):
        images, labels = data[0].to(device), data[1].to(device)

        with torch.no_grad():
            outputs = model(images)
            correct = sum(labels == torch.argmax(outputs, 1)).item()
            running_accuracy += correct/batch_size

    avg_acc_across_batches = (running_accuracy / len(dataloader)) * 100
    print('Accuracy on test set: {:.2f}%'.format(avg_acc_across_batches))
    return avg_acc_across_batches


def run_epochs(num_epochs, model, optimizer, loss_fn, pride=0.0, max_strikes=5):
    """
    An entire training session.
    """
    strike = 0 # how many time in a row the accuracy can decrease before stopping

    for epoch in range(num_epochs):
        # Start epoch
        print(f"Epoch {epoch+1} begin: {dt.now()}.")
        start = time()

        # Train the model
        train(model, train_loader, optimizer, loss_fn)
        print('Completed training after {:.2f} seconds.'.format((time() - start)))
        check_point = time()

        # Test the model
        acc = accuracy(model, test_loader)
        print('Completed testing after {:.2f} seconds.'.format((time() - check_point)))

        # Output an update
        print('Total epoch time: {:.2f} seconds.'.format((time() - start)))
        print('Epoch [{}/{}]'.format(epoch+1, num_epochs))
        check_point = time()

        # Update emotional state
        if pride < acc:
            pride = acc
            strike = 0
            print("Check-point!")
            torch.save(model.state_dict(), "./checkpoint.pth")

        else:
            strike += 1
            if strike >= max_strikes:
                print("The model is not improving!")
                torch.save(model.state_dict(), "./endpoint.pth")
                break

        print("####################################################################################################")
    
    

In [18]:
### TRAINING ###

# Create model object and connect it to the corresponding processing
model = CNN().to(device)

# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# Define mental state and start
peak_accuracy, max_strikes = 0.0, 5
run_epochs(num_epochs, model, optimizer, loss_fn, peak_accuracy, max_strikes=5)

Epoch 1 begin: 2025-05-16 12:58:09.072842.


KeyboardInterrupt: 

In [19]:
model, random_seed, batch_size, lr, momentum

(CNN(
   (layer1): Sequential(
     (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
     (1): ReLU()
     (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   )
   (layer2): Sequential(
     (0): Block(
       (layer1): Sequential(
         (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
         (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (2): SiLU()
       )
       (layer2): Sequential(
         (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
         (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (1): Block(
       (layer1): Sequential(
         (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
         (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

# Calculating model performance
Load the best version of your model ( which should be produced and saved by previous cells ), calculate and report the test accuracy.

In [20]:
### MODEL COMPARISONS ###

# Load the best model weights
model = CNN().to(device)
model.load_state_dict(torch.load("checkpoint.pth", weights_only=True))

# Calculate and present final scores
final_test_acc = accuracy(model, test_loader)
print(f"Final Test Accuracy: {final_test_acc:.2f}%")


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint.pth'

# Summary of hyperparameters
Report the hyperparameters ( learning rate etc ) that you used in your final model for reproducibility.

# Simulation of random user
Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt

In [ ]:
# Your code here
# Below an example showing the format of the code output

# Bonus point
Use an LLM (API) to generate a description of the food preference of a user based on 10 images that a potential user could provide. 
Please include an example of the output of your code, especially if you used an API other than the OpenAI API.

This should work well even with differing test images by setting different random seeds for the image selector.